Data Collection [20 marks]:
● Code a wikipedia crawler and collect a large and diverse textual dataset
suitable for training word embeddings. (10 marks)
● Recommended sources: Wikipedia dumps, Project Gutenberg, news
articles, etc.
● Ensure that the dataset is preprocessed: remove special characters,
lowercase all words, etc. (10 marks)
3. Training [20 marks]:
● Use a Word2Vec embeddings technique. (10 marks)
● Utilise Gensim library to assist with the training.
● Save the trained model for future use. (10 marks)


In [1]:
!pip install wikipedia-api
!pip install contractions


In [ ]:
import wikipediaapi #for accessing wikipedia content
import nltk #Imports Natural Language Toolkit (NLTK) for working with human language data
import unicodedata #for processing and normalizing Unicode strings
import gensim #open-source library for unsupervised topic modeling and natural language processing
import re #Python's built-in package for regular expressions
from nltk.corpus import stopwords #imports`stopwords` function from NLTK.
from nltk.tokenize import word_tokenize #imports function for splitting string into list of words
from nltk.tokenize import sent_tokenize #imports function for splitting string into list of sentences.
import contractions #module for exapanding contractions
nltk.download('punkt') #prerequisite for NLTK's word and sentence tokenizer functions
nltk.download('stopwords') #downloads a set of common stopwords
    
user = 'WKDcodingassignment/1.0 (wk3218pgt@students.nulondon.ac.uk)' #user identifier for Wikipedia requests
wiki_wiki = wikipediaapi.Wikipedia(user, 'en') #initialize Wikipedia API in English
api_key = input("Please enter your Guardian API key:") #prompt for Guardian API key!!!!!!!!!!!!

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\willi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\willi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def get_category_pages_content(category_titles): #fetches specified Wikipedia content and combines pages into single string
    combined_text = ""

    for category_title in category_titles: #check if a category page
        category_page = wiki_wiki.page("Category:" + category_title)
        if category_page.exists():  #if a category page iterate over its members and fetch articles
            for cat in category_page.categorymembers.values():
                if cat.ns == wikipediaapi.Namespace.MAIN:
                    combined_text += cat.text #concatenate text of category articles
        else:  #otherwise handle as a regular Wikipedia article
            article_page = wiki_wiki.page(category_title)
            if article_page.exists():
                combined_text += article_page.text #and add article to combined text

    return combined_text

category_titles = ["Computer Science", "Filtration", "Economy of Italy", "The Addams Family (musical)", "American Civl War", "Football", "Sadness", "Amazons", "Ice bath", "Solstice", "Age of Aquarius", "Western culture", "Tropics", "Kinship", "Political spectrum", "Boy or girl paradox", "Hot and cold cognition", "Season", "Girls & Boys", "Temperature", "Global North and Global South", "Chinese law", "Central heating", "East End of London", "East–West dichotomy", "Short(finance)", "Body weight", "Thermostat", "Compass", "Cat–dog relationship", "State of matter", "Advertising", "Common sense", "United Nations", "Mr. Men", "Night", "Learning environment", "Rain", "Fire", "Seven deadly sins", "Emotion", "White", "Art", "Pets", "Opposite (semantics)", "Adjectives", "Family", "Money", "Work (human activity)", "Health", "Predation", "Culture", "Biology", "Charles Dickens", "A Vindication of the Rights of Woman", "Grammatical gender", "Presidency of Donald Trump", "Feminism", "Nelson Mandela", "Hip hop music", "Mexico", "Eastenders", "Porn", "Modern era", "Millenials", "Student–teacher ratio", "Cat people and dog people", "Global cuisine", "Earth", "Artificial intelligence", "Sport", "English language", "Opposite(semantics)", "Nature", "Human","Ageing", "Black"]
#my list of categories
wiki_category_content = get_category_pages_content(category_titles) #compiles and stores Wikipedia content from listed categories

In [4]:
len(wiki_category_content) #total number of characters in captured wikipedia content

26443706

In [5]:
!pip install requests
import requests #necessary for making HTTP requests

def fetch_guardian_articles(api_key, queries): #define function to fetch articles from Guardian API
    Guardian_url = "https://content.guardianapis.com/search"
    articles_text = ""  #initialise variable to hold combined article text
    for query in queries: #iterate over each query term
     
        params = {                           #sets up parameters for Guardian request
            'api-key': api_key,              #API key for authentication
            'q': query,                      #query term
            'page-size': 200,                #number of articles to fetch
            'show-fields': 'body'            #specifies that only main body of articles to be returned
        }
        
        response = requests.get(Guardian_url, params=params)    #make API request
        data = response.json() #convert JSON response into Python

        for article in data['response']['results']: #Guardian safety check/extract article bodies and add to combined text
            if 'body' in article['fields']:
                articles_text += article['fields']['body']
    
    return articles_text    #return the combined text of all articles
query_terms = ["Culture", "Sport", "Lifestyle", "Obituaries", "Corrections and clarifications", "Home", "Opinion", "Film", "Business", "Society"]
#specified query terms
guardian_content = fetch_guardian_articles(api_key, query_terms) #compiles and stores Guardian content from listed categories

In [6]:
len (guardian_content) #number of charcters in retrieved Guardian content

15054392

In [7]:
print(guardian_content[:500]) #check first 500 characters

<p>Your editorial highlights a problem that afflicts not just the arts but the culture sector as a whole (<a href="https://www.theguardian.com/commentisfree/2023/dec/22/the-guardian-view-on-women-in-art-a-cycle-of-here-today-gone-tomorrow-must-be-halted">The Guardian view on women in art: a cycle of here today, gone tomorrow must be halted, 22 December</a>).</p> <p>Successive surveys have identified “significant inequalities” and discrimination in the sector – not only in terms of gender but als


In [8]:
def remove_html_tags(text):  #defined function for removing html in a way that does not concatenate words separated by these tags
    return re.sub(r'<.*?>', ' ', text) #replaces all html tags with single space with re module 

cleaned_guardian_content = remove_html_tags(guardian_content) #remove html tags
cleaned_guardian_content = cleaned_guardian_content.strip() #remove trailing and white spaces

In [9]:
print(cleaned_guardian_content[:500]) #check first 500 characters

Your editorial highlights a problem that afflicts not just the arts but the culture sector as a whole ( The Guardian view on women in art: a cycle of here today, gone tomorrow must be halted, 22 December ).   Successive surveys have identified “significant inequalities” and discrimination in the sector – not only in terms of gender but also class, ethnicity, disability etc. The pandemic and cost of living crisis have deepened this imbalance. Arts and culture are in danger of becoming the preserv


In [10]:
def preprocessed_text(text):
    text = contractions.fix(text)  #expand contractions
    text = text.lower()  #lowercase characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')  #remove non-ASCII characters
    text = text.replace ("-"," ") #removes hyphens and adds space
    text = text.replace('nights','night').replace('africas', 'africa').replace ('color', 'colour').replace('nugent', 'student').replace('womans', 'woman').replace('easts', 'east').replace('liquids','liquid').replace('solids','solid').replace('evening','').replace('mid', '').replace('dsm', 'physical').replace('cheaper', 'cheap').replace('newcomer','boy').replace('vapor','gas').replace('mans','man').replace('fathers','father').replace('bulb','dry')
    #bespoke removal of some words including plurals and US spellings
    return text

cleaned_guardian_text = preprocessed_text(cleaned_guardian_content) #clean both datasets using preprocessed_text defined function
cleaned_wiki_text = preprocessed_text(wiki_category_content)
combined_content = cleaned_guardian_text + " " + cleaned_wiki_text #combine the two datasets

def tokenise_and_clean(text):
    stop_words = set(stopwords.words('english')) #initialises a set of stopwords from NLTK
    sentences = sent_tokenize(text) #splits text into individual sentences to process on a sentence-by-sentence basis
    cleaned_sentences = []
    for sentence in sentences:
        tokens = word_tokenize(sentence) #breaks each sentence into individual words.
        cleaned_tokens = [token for token in tokens if token.isalpha() and token not in stop_words] #filters out non-alphabetic characters (again, just in case)
        cleaned_sentences.append(cleaned_tokens) #appends cleaned list of word tokens to cleaned_sentences list
    return cleaned_sentences #returns list of sentence lists with cleaned word tokens
 
final_dataset = tokenise_and_clean(combined_content) #tokenise combined dataset using defined function

In [11]:
print(combined_content[:500]) #check first 500 characters

your editorial highlights a problem that afflicts not just the arts but the culture sector as a whole ( the guardian view on women in art: a cycle of here today, gone tomorrow must be halted, 22 december ).   successive surveys have identified significant inequalities and discrimination in the sector  not only in terms of gender but also class, ethnicity, disability etc. the pandemic and cost of living crisis have deepened this imbalance. arts and culture are in danger of becoming the preserve o


In [12]:
print(final_dataset[:10]) 

[['editorial', 'highlights', 'problem', 'afflicts', 'arts', 'culture', 'sector', 'whole', 'guardian', 'view', 'women', 'art', 'cycle', 'today', 'gone', 'tomorrow', 'must', 'halted', 'december'], ['successive', 'surveys', 'identified', 'significant', 'inequalities', 'discrimination', 'sector', 'terms', 'gender', 'also', 'class', 'ethnicity', 'disability', 'etc'], ['pandemic', 'cost', 'living', 'crisis', 'deepened', 'imbalance'], ['arts', 'culture', 'danger', 'becoming', 'preserve', 'white', 'public', 'school', 'educated', 'men'], ['change', 'must', 'start', 'top', 'ensure', 'incredibly', 'diverse', 'culture', 'fully', 'represented', 'museums', 'galleries', 'theatres'], ['governing', 'bodies', 'leading', 'arts', 'institutions', 'longer', 'reflect', 'narrow', 'elite', 'within', 'society'], ['inequality', 'also', 'symptomatic', 'much', 'wider', 'problem'], ['despite', 'contributing', 'economy', 'sector', 'underfunded', 'comparison', 'france', 'germany'], ['increasing', 'public', 'funding',

In [13]:
len(final_dataset)

229656

In [1172]:
model = gensim.models.Word2Vec(vector_size=100, min_count=22, window=14, sg=0)
model.build_vocab(final_dataset, update=False) #Build vocabulary from the dataset
model.train(final_dataset, total_examples=model.corpus_count, epochs=12) #train the model
model.save("Assignment2_FINAL.model") #save trained model



In [1173]:
print(model.wv.most_similar('dog', topn=5))
print(model.wv.most_similar('cat', topn=5))

[('cat', 0.7450248599052429), ('dogs', 0.7199230194091797), ('breeds', 0.6953954696655273), ('mouse', 0.6488680243492126), ('rat', 0.6244873404502869)]
[('dog', 0.7450249195098877), ('cats', 0.7063226699829102), ('breeds', 0.6797534227371216), ('feral', 0.6685229539871216), ('bird', 0.657339334487915)]


In [1174]:
print(model.wv.most_similar('woman', topn=5))
print(model.wv.most_similar('man', topn=5))

[('man', 0.6947796940803528), ('girl', 0.6297411322593689), ('wife', 0.582608699798584), ('daughter', 0.5816055536270142), ('husband', 0.5711905360221863)]
[('woman', 0.6947798132896423), ('devil', 0.5759873390197754), ('heaven', 0.5567798614501953), ('mankind', 0.548254132270813), ('sacrifices', 0.5291836261749268)]


In [1175]:
print(model.wv.most_similar('west', topn=5))
print(model.wv.most_similar('east', topn=5))

[('east', 0.6627454161643982), ('lands', 0.6210088729858398), ('asia', 0.6152318716049194), ('indies', 0.5967389345169067), ('anglia', 0.5903019905090332)]
[('west', 0.662745475769043), ('eastern', 0.5534988045692444), ('asia', 0.5445843935012817), ('lands', 0.5419663786888123), ('southeast', 0.5314397215843201)]


In [1176]:
print(model.wv.most_similar('student', topn=5))
print(model.wv.most_similar('teacher', topn=5))

[('teacher', 0.7772083878517151), ('students', 0.7516104578971863), ('graduate', 0.6935057044029236), ('college', 0.6864336133003235), ('school', 0.6573134660720825)]
[('student', 0.7772082686424255), ('school', 0.7160865068435669), ('graduating', 0.7095962166786194), ('grammar', 0.6736576557159424), ('taught', 0.6620268225669861)]


In [1177]:
print(model.wv.most_similar('south', topn=5))
print(model.wv.most_similar('north', topn=5))

[('north', 0.609523594379425), ('mandela', 0.5427833795547485), ('scramble', 0.528521716594696), ('saharan', 0.5193409323692322), ('zimbabwe', 0.517381489276886)]
[('south', 0.6095237135887146), ('peninsula', 0.5358458161354065), ('northwest', 0.5203916430473328), ('coast', 0.5144932270050049), ('southern', 0.5097523331642151)]


In [1178]:
print(model.wv.most_similar('closed', topn=5))
print(model.wv.most_similar('open', topn=5))

[('doors', 0.6430973410606384), ('door', 0.629950761795044), ('open', 0.563855767250061), ('shut', 0.5577067136764526), ('offices', 0.5560802817344666)]
[('closed', 0.5638558268547058), ('door', 0.45219311118125916), ('opening', 0.43385207653045654), ('doors', 0.43113845586776733), ('opened', 0.3778987228870392)]


In [1179]:
print(model.wv.most_similar('long', topn=5))
print(model.wv.most_similar('short', topn=5))

[('short', 0.7532863616943359), ('longer', 0.527293860912323), ('longest', 0.46232086420059204), ('coined', 0.3931172788143158), ('full', 0.3702988624572754)]
[('long', 0.7532865405082703), ('coined', 0.46949952840805054), ('shortened', 0.4481065273284912), ('full', 0.439769983291626), ('shorter', 0.4356118142604828)]


In [1180]:
print(model.wv.most_similar('sad', topn=5))
print(model.wv.most_similar('happy', topn=5))

[('happy', 0.7386326789855957), ('angry', 0.7276406288146973), ('depressed', 0.7238308787345886), ('fearful', 0.7153260707855225), ('funny', 0.7120686173439026)]
[('sad', 0.7386327385902405), ('watching', 0.6454654335975647), ('ok', 0.6271947622299194), ('happily', 0.5901420712471008), ('cheer', 0.588717520236969)]


In [1181]:
print(model.wv.most_similar('wife', topn=5))
print(model.wv.most_similar('husband', topn=5))

[('husband', 0.8701814413070679), ('daughter', 0.7434009313583374), ('housekeeper', 0.7133551836013794), ('widow', 0.7086474299430847), ('mistress', 0.6997210383415222)]
[('wife', 0.8701815009117126), ('daughter', 0.7599546909332275), ('widow', 0.7529945373535156), ('divorced', 0.7044995427131653), ('daughters', 0.7011381983757019)]


In [1182]:
print(model.wv.most_similar('solid', topn=5))
print(model.wv.most_similar('liquid', topn=5))

[('liquid', 0.8006709814071655), ('atoms', 0.7868486046791077), ('porous', 0.7759988307952881), ('crystals', 0.7748749852180481), ('combustible', 0.7698103189468384)]
[('solid', 0.800670862197876), ('combustible', 0.7982779145240784), ('flammable', 0.789177417755127), ('droplets', 0.7715269923210144), ('helium', 0.7707858681678772)]


In [1183]:
print(model.wv.most_similar('hot', topn=5))
print(model.wv.most_similar('cold', topn=5))

[('cool', 0.7853217124938965), ('warm', 0.6869105696678162), ('boil', 0.6822919249534607), ('cooler', 0.6769410371780396), ('moist', 0.6747627258300781)]
[('hot', 0.6414405703544617), ('cool', 0.5339387655258179), ('warm', 0.5283296704292297), ('cooler', 0.5203039646148682), ('colder', 0.5055597424507141)]


In [1184]:
print(model.wv.most_similar('summer', topn=5))
print(model.wv.most_similar('winter', topn=5))

[('winter', 0.7788777947425842), ('spring', 0.6924644708633423), ('season', 0.6415839791297913), ('holidays', 0.6089718341827393), ('autumn', 0.5982123613357544)]
[('summer', 0.7788777351379395), ('spring', 0.7587435245513916), ('rainy', 0.6934080123901367), ('seasonal', 0.640884280204773), ('olympics', 0.6272353529930115)]


In [1185]:
print(model.wv.most_similar('boy', topn=5))
print(model.wv.most_similar('girl', topn=5))

[('girl', 0.7922379970550537), ('heron', 0.6655763983726501), ('mia', 0.6218944191932678), ('bb', 0.6123096942901611), ('kid', 0.6094543933868408)]
[('boy', 0.7922379970550537), ('kid', 0.666678249835968), ('mia', 0.6488155126571655), ('teenager', 0.638354480266571), ('kansara', 0.6349530816078186)]


In [1186]:
print(model.wv.most_similar('father', topn=5))
print(model.wv.most_similar('mother', topn=5))

[('mother', 0.7913547158241272), ('son', 0.6851228475570679), ('mothers', 0.6382254958152771), ('husband', 0.6368588209152222), ('daughter', 0.6168530583381653)]
[('father', 0.7913545370101929), ('daughter', 0.694534182548523), ('mothers', 0.6765509247779846), ('grandmother', 0.6589463353157043), ('son', 0.6299551129341125)]


In [1187]:
print(model.wv.most_similar('dry', topn=5))
print(model.wv.most_similar('wet', topn=5))

[('wet', 0.851753830909729), ('cool', 0.6813039183616638), ('hot', 0.6475218534469604), ('moist', 0.6428397297859192), ('cooler', 0.6387968063354492)]
[('dry', 0.851753830909729), ('rainy', 0.6768277287483215), ('moist', 0.6737340688705444), ('drier', 0.6651470065116882), ('climates', 0.6649980545043945)]


In [1188]:
print(model.wv.most_similar('weak', topn=5))
print(model.wv.most_similar('strong', topn=5))

[('strong', 0.6458708643913269), ('perceived', 0.5351082682609558), ('lacking', 0.5304543375968933), ('tendency', 0.527560830116272), ('contrast', 0.5169390439987183)]
[('weak', 0.6458709239959717), ('stronger', 0.6076570153236389), ('moderate', 0.4794518053531647), ('tendency', 0.43355831503868103), ('low', 0.4282933175563812)]


In [1189]:
print(model.wv.most_similar('low', topn=5))
print(model.wv.most_similar('high', topn=5))

[('high', 0.8159922957420349), ('lower', 0.6038392186164856), ('higher', 0.6007924675941467), ('elevated', 0.5947992205619812), ('poorer', 0.5706573724746704)]
[('low', 0.8159922957420349), ('higher', 0.5829386115074158), ('elevated', 0.5755956768989563), ('lower', 0.5274057984352112), ('highest', 0.5017989873886108)]


In [1190]:
print(model.wv.most_similar('day', topn=5))
print(model.wv.most_similar('night', topn=5))

[('night', 0.5771267414093018), ('armistice', 0.4831666648387909), ('week', 0.4533112347126007), ('hours', 0.44787007570266724), ('hour', 0.4469852149486542)]
[('twilight', 0.5885431170463562), ('afternoon', 0.5856591463088989), ('morning', 0.5819016695022583), ('day', 0.5771268010139465), ('sunset', 0.575907826423645)]


In [1191]:
print(model.wv.most_similar('horizontal', topn=5))
print(model.wv.most_similar('vertical', topn=5))

[('vertical', 0.8936328291893005), ('angle', 0.8119324445724487), ('rotating', 0.799260675907135), ('perpendicular', 0.7755590081214905), ('spheroid', 0.771782636642456)]
[('horizontal', 0.8936327695846558), ('curvature', 0.7800136804580688), ('angle', 0.7786969542503357), ('rotating', 0.7739986181259155), ('layer', 0.7514842748641968)]


In [1192]:
print(model.wv.most_similar('expensive', topn=5))
print(model.wv.most_similar('cheap', topn=5))

[('cheap', 0.8360576033592224), ('costly', 0.6451006531715393), ('efficient', 0.6404746770858765), ('inexpensive', 0.6355885863304138), ('rents', 0.631320059299469)]
[('expensive', 0.8360574841499329), ('supermarkets', 0.7259688973426819), ('vegetables', 0.6981269717216492), ('sell', 0.6708320379257202), ('meat', 0.6679823994636536)]


In [1193]:
print(model.wv.most_similar('right', topn=5))
print(model.wv.most_similar('left', topn=5))

[('left', 0.69141685962677), ('wing', 0.6004259586334229), ('leaning', 0.5464115738868713), ('far', 0.47612857818603516), ('populists', 0.4649846851825714)]
[('right', 0.69141685962677), ('wing', 0.6454244256019592), ('leaning', 0.6045607328414917), ('far', 0.5697712898254395), ('liberals', 0.5219332575798035)]


In [1194]:
print(model.wv.most_similar('black', topn=5))
print(model.wv.most_similar('white', topn=5))

[('colour', 0.6311914324760437), ('hispanic', 0.5396310091018677), ('blacks', 0.5295069813728333), ('yellow', 0.5064201951026917), ('collar', 0.4985024034976959)]
[('black', 0.4631962478160858), ('colour', 0.46016258001327515), ('collar', 0.42550158500671387), ('representatives', 0.42286020517349243), ('blacks', 0.4176049828529358)]


In [1195]:
print(model.wv.most_similar('bottom', topn=5))
print(model.wv.most_similar('top', topn=5))

[('top', 0.6775108575820923), ('vertical', 0.6071323752403259), ('spinning', 0.5627757906913757), ('horizontal', 0.557688295841217), ('gear', 0.5531451106071472)]
[('bottom', 0.6775107979774475), ('tier', 0.6058933138847351), ('rankings', 0.5353025794029236), ('super', 0.5241504311561584), ('kane', 0.5166078805923462)]


In [1204]:
print(model.wv.most_similar('blue', topn=5))
print(model.wv.most_similar('green', topn=5))

[('carpet', 0.7061049342155457), ('yellow', 0.6860446929931641), ('orange', 0.6776986718177795), ('bright', 0.6708247661590576), ('red', 0.6602861285209656)]
[('bethnal', 0.6114839911460876), ('blue', 0.6064355373382568), ('carpet', 0.5596210956573486), ('yellow', 0.5187549591064453), ('bowling', 0.5135565400123596)]


In [1197]:
print(model.wv.most_similar('bad', topn=5))
print(model.wv.most_similar('good', topn=5))

[('good', 0.655204176902771), ('surely', 0.5798863768577576), ('wrong', 0.5378819108009338), ('obviously', 0.5340209603309631), ('irresponsible', 0.531457245349884)]
[('bad', 0.655204176902771), ('deeds', 0.5415806770324707), ('luck', 0.5196453332901001), ('surely', 0.4851793348789215), ('better', 0.46380510926246643)]


In [1198]:
print(model.wv.most_similar('early', topn=5))
print(model.wv.most_similar('late', topn=5))

[('late', 0.7005549669265747), ('twentieth', 0.6408651471138), ('nineteenth', 0.6021789312362671), ('sixteenth', 0.5912802815437317), ('onwards', 0.5807916522026062)]
[('early', 0.7005549073219299), ('twentieth', 0.6306740641593933), ('onwards', 0.6257380843162537), ('nineteenth', 0.5918053984642029), ('sixteenth', 0.5695647597312927)]


In [1199]:
print(model.wv.most_similar('ceiling', topn=5))
print(model.wv.most_similar('floor', topn=5))

[('floor', 0.7275838851928711), ('gutter', 0.7120457887649536), ('feet', 0.7026998996734619), ('pipes', 0.7020912766456604), ('roof', 0.6793196797370911)]
[('roof', 0.7759767174720764), ('entrance', 0.7694216966629028), ('feet', 0.7525196671485901), ('ceiling', 0.7275838851928711), ('beside', 0.7157697677612305)]


In [1200]:
import os
print ("Model saved in:", os.getcwd())

Model saved in: C:\Users\willi
